## Note: all challenges have been directly copied from cryptopals.com.

In [1]:
import codecs
from collections import Counter
import string
from Crypto.Cipher import AES
from Crypto import Random
import random as rnd
import time
import hashlib
import requests
import os

# those variables are used later in several problems so I just initialize them here.
secret_key = Random.get_random_bytes(16)
iv_secret = Random.get_random_bytes(16)
secret_key_16_bits = Random.get_random_bytes(2)

Copied from cryptopals.com
## Crypto Challenge Set 1

This is the qualifying set. We picked the exercises in it to ramp developers up gradually into coding cryptography, but also to verify that we were working with people who were ready to write code.

This set is relatively easy. With one exception, most of these exercises should take only a couple minutes. But don't beat yourself up if it takes longer than that. It took Alex two weeks to get through the set!

If you've written any crypto code in the past, you're going to feel like skipping a lot of this. Don't skip them. At least two of them (we won't say which) are important stepping stones to later attacks. 


## 1-Convert hex to base64

The string:
```
49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d
```
Should produce:
```
SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t
```
So go ahead and make that happen. You'll need to use this code for the rest of the exercises.

Cryptopals Rule
```
Always operate on raw bytes, never on encoded strings. Only use hex and base64 for pretty-printing.
```

In [2]:
## Challenge 1 convert from hex to base64
## This one is pretty straightforward
input = "49276d206b696c6c696e6720796f757220627261696e206c696b65206120706f69736f6e6f7573206d757368726f6f6d"
raw = codecs.decode(input, "hex")
result = codecs.encode(raw,"base64")
expected_result = "SSdtIGtpbGxpbmcgeW91ciBicmFpbiBsaWtlIGEgcG9pc29ub3VzIG11c2hyb29t"
result.strip().decode('ascii') == expected_result

True


## 2-Fixed XOR

Write a function that takes two equal-length buffers and produces their XOR combination.

If your function works properly, then when you feed it the string:

1c0111001f010100061a024b53535009181c

... after hex decoding, and when XOR'd against:

686974207468652062756c6c277320657965

... should produce:

746865206b696420646f6e277420706c6179



In [3]:
## Challenge 2 FIxed XOR
input1 = codecs.decode("1c0111001f010100061a024b53535009181c","hex")
input2 = codecs.decode("686974207468652062756c6c277320657965","hex")

def xor_string(string1,string2):
    """ Receive two bytes strings and return the xor in them
        The length of the output is the same as the shortest string
    """
    return bytes(a ^ b for a, b in zip(string1, string2))

expected_result = "746865206b696420646f6e277420706c6179"

codecs.encode(xor_string(input1,input2),"hex").strip().decode('ascii') == expected_result

True

## 3-Single-byte XOR cipher

The hex encoded string:

1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736

... has been XOR'd against a single character. Find the key, decrypt the message.

You can do this by hand. But don't: write code to do it for you.

How? Devise some method for "scoring" a piece of English plaintext. Character frequency is a good metric. Evaluate each output and choose the one with the best score.

Achievement Unlocked

You now have our permission to make "ETAOIN SHRDLU" jokes on Twitter.


In [4]:
## Challenge 3 xor single byte
input1 = codecs.decode("1b37373331363f78151b7f2b783431333d78397828372d363c78373e783a393b3736","hex")

#Here I'm using as an score of a piece of English plaintext not the character frequency but rather the fact
#that the correct english phrase is more likely to have the most amount of words, this isn't valid in all cases,but worked in this one
biggest = 0;
biggest_character = '';
for character in range(0,128):
    output = xor_string(input1,bytes(chr(character)*len(input1),'ascii'))
    if len(output.decode('ascii').split(' '))>biggest:
        biggest = len(output.decode('ascii').split(' '))
        biggest_character = character
        
output = xor_string(input1,bytes(chr(biggest_character)*len(input1),'ascii'))
output

b"Cooking MC's like a pound of bacon"

## 4-Detect single-character XOR

One of the 60-character strings in this file has been encrypted by single-character XOR.

Find it.

(Your code from #3 should help.)

In [5]:
## Challenge 4 find encoded string in file

lines = open('4.txt', 'r').readlines()

## I'm gonna use the same logic from the 3rd problem and see the line with the most amont of words
longest_line =""
global_longest = 0;

for line in lines:
    line_raw = codecs.decode(line.strip(),'hex')
    biggest = 0;
    biggest_character = 0;
    for character in range(0,128):
        output = xor_string(line_raw,bytes(len(line_raw)*chr(character),'ascii'))
        flag = False;
        for i in output:
            if i>=128: # if output is not in ascii then it's not an english sentence
                flag = True;
        
        if flag:
            continue
        
        if len(output.decode('ascii').split(' '))>biggest: ## compare between characters in the line
            biggest = len(output.decode('ascii').split(' '))
            biggest_character = character;
            
        if biggest>global_longest: ## compare between all lines
            global_longest=biggest;
            longest_line = output;

print(longest_line)

b'Now that the party is jumping\n'


## 5-Implement repeating-key XOR

Here is the opening stanza of an important work of the English language:
```
Burning 'em, if you ain't quick and nimble
I go crazy when I hear a cymbal
```
Encrypt it, under the key "ICE", using repeating-key XOR.

In repeating-key XOR, you'll sequentially apply each byte of the key; the first byte of plaintext will be XOR'd against I, the next C, the next E, then I again for the 4th byte, and so on.

It should come out to:

0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272
a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f

Encrypt a bunch of stuff using your repeating-key XOR function. Encrypt your mail. Encrypt your password file. Your .sig file. Get a feel for it. I promise, we aren't wasting your time with this.


In [6]:
## Challenge 5 repeating key xor

#def repeating_xor(input1,key):
#    output = b''
#    while len(input1)>0:
#        output += xor_string(input1,key) 
#        input1 = input1[len(key):]
#    return output

# this one is a little bit faster, although the other function is easier to read and uses previous functions
def repeating_xor(input1,key):
    output=[]
    for s in range(0,len(input1)):
        output.append(input1[s] ^ key[s % len(key)])
    return bytes(output)


input1 = "Burning 'em, if you ain't quick and nimble"
key = "ICE"
input2 = "I go crazy when I hear a cymbal"

input3 = input1+ '\n' +input2;
result1 = repeating_xor(bytes(input1,'ascii'),bytes(key,'ascii'))
result2 = repeating_xor(bytes(input2,'ascii'),bytes(key,'ascii'))
result3 = repeating_xor(bytes(input3,'ascii'),bytes(key,'ascii'))

expected1 = "0b3637272a2b2e63622c2e69692a23693a2a3c6324202d623d63343c2a26226324272765272"
expected2 = "a282b2f20430a652e2c652a3124333a653e2b2027630c692b20283165286326302e27282f"

codecs.encode(result3,'hex').decode('ascii') == expected1 + expected2

True

## 6-Break repeating-key XOR
```
It is officially on, now.

This challenge isn't conceptually hard, but it involves actual error-prone coding. The other challenges in this set are there to bring you up to speed. This one is there to qualify you. If you can do this one, you're probably just fine up to Set 6.
```

There's a file here. It's been base64'd after being encrypted with repeating-key XOR.

Decrypt it.

Here's how:

1. Let KEYSIZE be the guessed length of the key; try values from 2 to (say) 40.
2. Write a function to compute the edit distance/Hamming distance between two strings. The Hamming distance is just the number of differing bits. The distance between:
```
    this is a test
```
    and
```
    wokka wokka!!!
```
is 37. Make sure your code agrees before you proceed.
3. For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit distance between them. Normalize this result by dividing by KEYSIZE.
4. The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
5. Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
6. Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
7. Solve each block as if it was single-character XOR. You already have code to do this.
8. For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.

This code is going to turn out to be surprisingly useful later on. Breaking repeating-key XOR ("Vigenere") statistically is obviously an academic exercise, a "Crypto 101" thing. But more people "know how" to break it than can actually break it, and a similar technique breaks something much more important.
```
No, that's not a mistake.

We get more tech support questions for this challenge than any of the other ones. We promise, there aren't any blatant errors in this text. In particular: the "wokka wokka!!!" edit distance really is 37.
```

In [7]:
## Challenge 6. Break a repeating xor key( aka Vigenere cypher)
## this is the most challeging exercise so far, but they broke it down step by step so lets follow them

# 2 - Write a function to compute the edit distance/Hamming distance between two strings
# There are probably faster applications of this function in wikipedia
def hamming_distance(bytes1,bytes2):
    after_xor = xor_string(bytes1,bytes2); #xor so that the different bits will became '1'
    return bin(int.from_bytes(after_xor,'little')).count('1') #count the number of '1' bits on the resulting string

#test is the hamming function works as intended;
input1 = "this is a test"
input2 = "wokka wokka!!!"
assert hamming_distance(bytes(input1,'ascii'),bytes(input2,'ascii')) == 37

raw_file = open('6.txt','rb').read()
raw_file_decoded = codecs.decode(raw_file, "base64")

#3- For each KEYSIZE, take the first KEYSIZE worth of bytes, and the second KEYSIZE worth of bytes, and find the edit 
#distance between them. Normalize this result by dividing by KEYSIZE.
keysizes = [] #it's gonna be an array of tuples
for keysize in range(2,41):
    distances = []
    pieces = [raw_file_decoded[i:i+keysize] for i in range(0,len(raw_file_decoded),keysize)] #split the file in pieces of size keysize
    
    for i in range(0,len(pieces),2):
        if i+1<len(pieces): #if there are at least two pieces remaining
            distances.append(hamming_distance(pieces[i],pieces[i+1])/keysize) # calculate hamming distance and normalize
    
    distance = sum(distances)/len(distances); #average
    keysizes.append((keysize,distance))
            
            
#4 - The KEYSIZE with the smallest normalized edit distance is probably the key. You could proceed perhaps with the
#smallest 2-3 KEYSIZE values. Or take 4 KEYSIZE blocks instead of 2 and average the distances.
keysizes_sorted = sorted(keysizes,key=lambda tup: tup[1])
keysizes_selected = [a[0] for a in keysizes_sorted[:4]]

#5- Now that you probably know the KEYSIZE: break the ciphertext into blocks of KEYSIZE length.
#6 -Now transpose the blocks: make a block that is the first byte of every block, and a block that is the second byte of every block, and so on.
def GetNthLetters(text, position, keysize,debug=False):
    builtarray = []
    for i in range(position ,len(text),keysize):
        builtarray.append(text[i]);
    return bytes(builtarray)

blocks = [] #blocks is a 3-dimension matrix in which de indices(i,j) point to a byte sequence

for i in range(0,len(keysizes_selected)):
    blocks.append([])
    for j in range(0,keysizes_selected[i]):
        blocks[i].append(GetNthLetters(raw_file_decoded,j,keysizes_selected[i]))
        
#7-Solve each block as if it was single-character XOR. You already have code to do this.

# my idea to find text based on number of words don't work anymore in this case.
# I got this function from https://laconicwolf.com/2018/06/05/cryptopals-challenge-4-detect-single-character-xor-encryption/ for letter frequency
def get_english_score(input_bytes):
    """Compares each input byte to a character frequency 
    chart and returns the score of a message based on the
    relative frequency the characters occur in the English
    language
    """

    # From https://en.wikipedia.org/wiki/Letter_frequency
    # with the exception of ' ', which I estimated.
    character_frequencies = {
        'a': .08167, 'b': .01492, 'c': .02782, 'd': .04253,
        'e': .12702, 'f': .02228, 'g': .02015, 'h': .06094,
        'i': .06094, 'j': .00153, 'k': .00772, 'l': .04025,
        'm': .02406, 'n': .06749, 'o': .07507, 'p': .01929,
        'q': .00095, 'r': .05987, 's': .06327, 't': .09056,
        'u': .02758, 'v': .00978, 'w': .02360, 'x': .00150,
        'y': .01974, 'z': .00074, ' ': .13000, '@': -50000,
        '^': -50000, '#': -50000, '&': -50000, '~': -50000
    }
    pontuacao = sum([character_frequencies.get(chr(byte), 0) for byte in input_bytes.lower()])
    return pontuacao

def find_letter_xor(input1,byte=False):
    """Find the single byte xor_cypher that is most likely to be correct
        if byte = True, the input is treated as an byte string, otherwise, like ascii text
    """
    highest_score = 0;
    highest_character = 0;
    loop = 128
    if byte:
        loop=256
        
    for character in range(0,loop):
        if byte:
            after_xor = xor_string(input1,len(input1)*int.to_bytes(character,1,'little'))
        else:
            after_xor = xor_string(input1,bytes(len(input1)*chr(character),'ascii'))
        score = get_english_score(after_xor);
        
        if score>highest_score:
            highest_score = score
            highest_character = character
    
    if byte:
        return highest_character
    
    return chr(highest_character)

#8-For each block, the single-byte XOR key that produces the best looking histogram is the repeating-key XOR key byte for that block. Put them together and you have the key.
keys = {} #array of strings
for i in range(0,len(keysizes_selected)):
    keys[i] = ""
    for j in range(0,keysizes_selected[i]):
        keys[i] += find_letter_xor(blocks[i][j]);
        
keys

{0: 'Terminator X: Bring the noise',
 1: '\x00\x00B\x1e\x04U\x00`H\x00e\x1ee\x1a\x01}v\x07VS\x06\x06\x0b|P',
 2: 'OBuiT\x00e\x00eNq\x00BlesNTOaVs\x00IYfTFRe\x11f`:et\x1aOsi',
 3: 'e\x16lNRrgOn\x07[R{e\x06lOa\x1c\x00\x06RiT\x11\x00\x06U`RnE\x06'}

## 7-AES in ECB mode

The Base64-encoded content in this file has been encrypted via AES-128 in ECB mode under the key
```
"YELLOW SUBMARINE".
```
(case-sensitive, without the quotes; exactly 16 characters; I like "YELLOW SUBMARINE" because it's exactly 16 bytes long, and now you do too).

Decrypt it. You know the key, after all.

Easiest way: use OpenSSL::Cipher and give it AES-128-ECB as the cipher.
```
Do this with code.

You can obviously decrypt this using the OpenSSL command-line tool, but we're having you get ECB working in code for a reason. You'll need it a lot later on, and not just for attacking ECB.
```

In [8]:
# Challenge 7 AES-128 in ecb mode 
# ECB is the mode where each block is encrypted/decrypted separately.
# This one is pretty straightforward, just got to use the AES object from the Crypto.cypher module
key = "YELLOW SUBMARINE"
algo = AES.new(bytes(key,'ascii'),AES.MODE_ECB)

raw_file = codecs.decode(open('7.txt','rb').read(),"base64")
print(algo.decrypt(raw_file).decode('ascii'))

I'm back and I'm ringin' the bell 
A rockin' on the mike while the fly girls yell 
In ecstasy in the back of me 
Well that's my DJ Deshay cuttin' all them Z's 
Hittin' hard and the girlies goin' crazy 
Vanilla's on the mike, man I'm not lazy. 

I'm lettin' my drug kick in 
It controls my mouth and I begin 
To just let it flow, let my concepts go 
My posse's to the side yellin', Go Vanilla Go! 

Smooth 'cause that's the way I will be 
And if you don't give a damn, then 
Why you starin' at me 
So get off 'cause I control the stage 
There's no dissin' allowed 
I'm in my own phase 
The girlies sa y they love me and that is ok 
And I can dance better than any kid n' play 

Stage 2 -- Yea the one ya' wanna listen to 
It's off my head so let the beat play through 
So I can funk it up and make it sound good 
1-2-3 Yo -- Knock on some wood 
For good luck, I like my rhymes atrocious 
Supercalafragilisticexpialidocious 
I'm an effect and that you can bet 
I can take a fly girl and make her wet. 


## 8-Detect AES in ECB mode

In this file are a bunch of hex-encoded ciphertexts.

One of them has been encrypted with ECB.

Detect it.

Remember that the problem with ECB is that it is stateless and deterministic; the same 16 byte plaintext block will always produce the same 16 byte ciphertext.


In [9]:
## Challenge 8 Detect AES-128 ecb in file
# This is pretty simple, given the property of ECB given above, it's safe to assume that in an long enough
# series of data an ECB encryption will eventually repeat some blocks. If we count the number of repeted blocks
# we can guess which cyphertext used ECB

def detect_repetitions(cyphertext,keysize):
    number_of_repetitions = 0
    for a in range(0,keysize):
        pieces = [cyphertext[(i+a):(i+keysize+a)] for i in range(0,len(cyphertext),keysize)]
        number_of_repetitions += (len(pieces) - len(set(pieces)))
    return number_of_repetitions

raw_file = open('8.txt','rb').readlines()
cyphertexts = [codecs.decode(cyphertext.strip(),'hex') for cyphertext in raw_file]

repetitions = []
for a in range(0,len(cyphertexts)):
    number_of_repetitions = detect_repetitions(cyphertexts[a],16)
    repetitions.append((a,number_of_repetitions))
    
repetitions_sorted = sorted(repetitions,key=lambda tup: tup[1],reverse=True)
# They haven't given a way to verify which one is the actual answer, but only one cyphertext
# had repetitions at all.
repetitions_sorted[:2]

[(132, 3), (0, 0)]

# Crypto Challenge Set 2

This is the first of several sets on block cipher cryptography. This is bread-and-butter crypto, the kind you'll see implemented in most web software that does crypto.

This set is relatively easy. People that clear set 1 tend to clear set 2 somewhat quickly.

Three of the challenges in this set are extremely valuable in breaking real-world crypto; one allows you to decrypt messages encrypted in the default mode of AES, and the other two allow you to rewrite messages encrypted in the most popular modes of AES. 

## 9-Implement PKCS#7 padding

A block cipher transforms a fixed-sized block (usually 8 or 16 bytes) of plaintext into ciphertext. But we almost never want to transform a single block; we encrypt irregularly-sized messages.

One way we account for irregularly-sized messages is by padding, creating a plaintext that is an even multiple of the blocksize. The most popular padding scheme is called PKCS#7.

So: pad any block to a specific block length, by appending the number of bytes of padding to the end of the block. For instance,
```
"YELLOW SUBMARINE"
```
... padded to 20 bytes would be:
```
"YELLOW SUBMARINE\x04\x04\x04\x04"
```


In [10]:
## Challenge 9 Implement PKCS#7 padding

def pkcs_padding(text,size):
    padding_size = size-len(text);
    if padding_size<0:
        raise ValueError('Block smaller than text')
    padding = padding_size*chr(padding_size);
    return text+bytes(padding,'ascii')

def pkcs_padding_long(text,size):
    """Apply pkcs padding to an string of any length
       Size is the blocksize.
    """
    pieces = [text[i:i+size] for i in range(0,len(text),size)]
    pieces[-1] = pkcs_padding(pieces[-1],size)
    return b''.join(pieces)
    
def pkcs_unpadding(text,size):
    padding_size = int(text[-1]);
    padding = text[(size-padding_size):];
    if len(set(padding))==1:
        return text[:(size-padding_size)];
    else:
        return text
        
def pkcs_unpadding_long(text,size):
    """Remove pkcs padding from an string of any length
        Size is the blocksize.
    """
    pieces = [text[i:i+size] for i in range(0,len(text),size)]
    pieces[-1] = pkcs_unpadding(pieces[-1],size)
    return b''.join(pieces)
    
input1 = "YELLOW SUBMARINE"
output1 = pkcs_padding_long(bytes(input1,'ascii'),20)
print(output1)
pkcs_unpadding_long(output1,20)

b'YELLOW SUBMARINE\x04\x04\x04\x04'


b'YELLOW SUBMARINE'

## 10 - Implement CBC mode

CBC mode is a block cipher mode that allows us to encrypt irregularly-sized messages, despite the fact that a block cipher natively only transforms individual blocks.

In CBC mode, each ciphertext block is added to the next plaintext block before the next call to the cipher core.

The first plaintext block, which has no associated previous ciphertext block, is added to a "fake 0th ciphertext block" called the initialization vector, or IV.

Implement CBC mode by hand by taking the ECB function you wrote earlier, making it encrypt instead of decrypt (verify this by decrypting whatever you encrypt to test), and using your XOR function from the previous exercise to combine them.

The file here is intelligible (somewhat) when CBC decrypted against "YELLOW SUBMARINE" with an IV of all ASCII 0 (\x00\x00\x00 &c)


>***Don't cheat***.
>
>***Do not use OpenSSL's CBC code to do CBC mode, even to verify your results***. What's the point of even doing this stuff if you aren't going to learn from it?


Note: This from now on block is not a part of the question, it's just an comment of mine.

Here are some diagrams that I found useful when making the code. They're from Wikipedia

![](cbc_encrypt.png)
![](cbc_decrypt.jpg)

In [11]:
## Challenge 10 Implement CBC mode

def cbc_encrypt_manual(cleartext,key,iv):
    pieces = [cleartext[i:i+len(key)] for i in range(0,len(cleartext),len(key))]
    pieces[-1] = pkcs_padding(pieces[-1],len(key));
    algo = AES.new(key,AES.MODE_ECB);

    previous = iv;
    cyphertext = b'';
    for i in range(0,len(pieces)):
        input1 = xor_string(pieces[i],previous);
        output1 = algo.encrypt(input1);
        previous = output1;
        cyphertext += output1
        
    return cyphertext

def cbc_decrypt_manual(cyphertext,key,iv,padding=True):
    pieces = [cyphertext[i:i+len(key)] for i in range(0,len(cyphertext),len(key))]
    pieces[-1] = pkcs_padding(pieces[-1],len(key));
    algoritmo = AES.new(key,AES.MODE_ECB);

    previous = iv;
    cleartext = b'';
    for i in range(0,len(pieces)):
        block_output = algoritmo.decrypt(pieces[i]);
        output = xor_string(block_output,previous);
        previous = pieces[i];
        cleartext += output
    
    if padding:
        return pkcs_unpadding_long(cleartext,len(key))
    else:
        return cleartext
        
raw_file = open('10.txt','rb').read()
raw_file = codecs.decode(raw_file,'base64')
key = b"YELLOW SUBMARINE"
iv = b'\x00'*len(key)

result = cbc_decrypt_manual(raw_file,key,iv)
print(result)

#this is just to test if my encryption function also works. 
test = b"Dancin', Dancin', Dancin'\nCome on, come on, come on everybody\nCome on, come on everybody\nCome on everybody, come on, come on everybody\nCome on, come on everybody\nLet's do this dance"
cbc_decrypt_manual(cbc_encrypt_manual(test,key,iv),key,iv)

b"I'm back and I'm ringin' the bell \nA rockin' on the mike while the fly girls yell \nIn ecstasy in the back of me \nWell that's my DJ Deshay cuttin' all them Z's \nHittin' hard and the girlies goin' crazy \nVanilla's on the mike, man I'm not lazy. \n\nI'm lettin' my drug kick in \nIt controls my mouth and I begin \nTo just let it flow, let my concepts go \nMy posse's to the side yellin', Go Vanilla Go! \n\nSmooth 'cause that's the way I will be \nAnd if you don't give a damn, then \nWhy you starin' at me \nSo get off 'cause I control the stage \nThere's no dissin' allowed \nI'm in my own phase \nThe girlies sa y they love me and that is ok \nAnd I can dance better than any kid n' play \n\nStage 2 -- Yea the one ya' wanna listen to \nIt's off my head so let the beat play through \nSo I can funk it up and make it sound good \n1-2-3 Yo -- Knock on some wood \nFor good luck, I like my rhymes atrocious \nSupercalafragilisticexpialidocious \nI'm an effect and that you can bet \nI can take 

b"Dancin', Dancin', Dancin'\nCome on, come on, come on everybody\nCome on, come on everybody\nCome on everybody, come on, come on everybody\nCome on, come on everybody\nLet's do this dance"

## 11- An ECB/CBC detection oracle

Now that you have ECB and CBC working:

Write a function to generate a random AES key; that's just 16 random bytes.

Write a function that encrypts data under an unknown key --- that is, a function that generates a random key and encrypts under it.

The function should look like:
```
encryption_oracle(your-input)
=> [MEANINGLESS JIBBER JABBER]
```
Under the hood, have the function append 5-10 bytes (count chosen randomly) before the plaintext and 5-10 bytes after the plaintext.

Now, have the function choose to encrypt under ECB 1/2 the time, and under CBC the other half (just use random IVs each time for CBC). Use rand(2) to decide which to use.

Detect the block cipher mode the function is using each time. You should end up with a piece of code that, pointed at a block box that might be encrypting ECB or CBC, tells you which one is happening.


In [12]:
## Challenge 11 An ECB/CBC detection oracle

# Random is from the Crypto package and generates random bytes that are cryptographically secure
# Alternatively, one can use os.urandom() to generate random bytes with the same standars, but 
# since I already imported module Crypto to use AES, I'm gonna use it
# rnd is the python random module. Although not cryptographically secure, it's good for simulations
def encryption_oracle(cleartext):
    key = Random.get_random_bytes(16);
    salt = Random.get_random_bytes(rnd.randint(5,10));
    pepper = Random.get_random_bytes(rnd.randint(5,10));
    iv = Random.get_random_bytes(16);
    
    cleartext = salt + cleartext + pepper;
    mode = ''
    if rnd.randint(1,2) == 2:
        cyphertext = cbc_encrypt_manual(cleartext,key,iv);
        mode = 'cbc';
    else:
        algoritmo = AES.new(key,AES.MODE_ECB);
        cyphertext = algoritmo.encrypt(pkcs_padding_long(cleartext,16));
        mode = 'ecb';
    #the oracle return both the cyphertext and the mode cause I can then verify it later
    return [cyphertext,mode]

def detect_ecb_or_cbc(cyphertext,keysize):
    #this is from the challenge 8
    if detect_repetitions(cyphertext,keysize)>0:
        return 'ecb'
    else:
        return 'cbc'

#test to see if works as intented. Since the text needs to be long I picked an not so random wikipedia article
text = open('11.txt','rb').read()
sucesses = 0;
failures = 0;
#I disabled it cause it takes a little time to run and I don't need to run this every time I open the notebook
enabled = False
if enabled:
    for i in range(0,100):
        output = encryption_oracle(text)
        repetitions = detect_repetitions(output[0],16);
        guess = detect_ecb_or_cbc(output[0],16);
        if output[1] == guess:
            sucesses+=1;
        else:
            failures+=1;
        
[sucesses,failures]

[0, 0]

## 12-Byte-at-a-time ECB decryption (Simple)

Copy your oracle function to a new function that encrypts buffers under ECB mode using a consistent but unknown key (for instance, assign a single random key, once, to a global variable).

Now take that same function and have it append to the plaintext, BEFORE ENCRYPTING, the following string:
```
Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkg
aGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBq
dXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUg
YnkK
```

>Spoiler alert.
>
>**Do not decode this string now. Don't do it.**

Base64 decode the string before appending it. Do not base64 decode the string by hand; make your code do it. The point is that you don't know its contents.

What you have now is a function that produces:
```
AES-128-ECB(your-string || unknown-string, random-key)
```
It turns out: you can decrypt "unknown-string" with repeated calls to the oracle function!

Here's roughly how:

1. Feed identical bytes of your-string to the function 1 at a time --- start with 1 byte ("A"), then "AA", then "AAA" and so on. Discover the block size of the cipher. You know it, but do this step anyway.
2. Detect that the function is using ECB. You already know, but do this step anyways.
3. Knowing the block size, craft an input block that is exactly 1 byte short (for instance, if the block size is 8 bytes, make "AAAAAAA"). Think about what the oracle function is going to put in that last byte position.
4. Make a dictionary of every possible last byte by feeding different strings to the oracle; for instance, "AAAAAAAA", "AAAAAAAB", "AAAAAAAC", remembering the first block of each invocation.
5. Match the output of the one-byte-short input to one of the entries in your dictionary. You've now discovered the first byte of unknown-string.
6. Repeat for the next byte.

>Congratulations.
>
>This is the first challenge we've given you whose solution will break real crypto. Lots of people know that when you encrypt something in ECB mode, you can see penguins through it. Not so many of them can *decrypt the contents of those ciphertexts*, and now you can. If our experience is any guideline, this attack will get you code execution in security tests about once a year.


In [13]:
## Challenge 12 Byte-at-a-time ECB decryption (Simple)
# This challenge although is Simple is pretty fun, and it helps a lot that they've already layered the step-by-step process

# First I do the oracle
def encryption_oracle_ecb(cleartext,key):
    text_to_append = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
    text_to_append = codecs.decode(bytes(text_to_append,'ascii'),'base64')
    cleartext+=text_to_append;
    algoritmo = AES.new(key,AES.MODE_ECB);
    cyphertext = algoritmo.encrypt(pkcs_padding_long(cleartext,16));
    
    return cyphertext

# 1 - First I found the size of the block. I feed the encryption_oracle one byte at the time and wait until the output change size from 1 block to 2 blocks
previous = len(encryption_oracle_ecb(b'',secret_key))
for i in range(1,120): # 120 is just an arbitrary maximum I chose
    input1 = b'A'*i;
    output = encryption_oracle_ecb(input1,secret_key) #secret_key has to stay consistent between interations, so that's why I declared as a global variable in the first block of code
    if len(output)!=previous: 
        size = len(output)-previous
        break
        
# 2 - I use the function of challenge 11 here, I already know the result, so I don't need to save the result
detect_ecb_or_cbc(encryption_oracle_ecb(b'A'*size*10,secret_key),size)

# this function goes through steps 3 to 6
def decrypt_block(size,block_index,padding):
    # note that this padding here has no relation with the pkcs padding
    shift = size*block_index;
    cleartext = b''
    for i in range(0,size): # do this for every byte in the block
        dictionary = {} # will store all the possible bytes and their result
        for j in range(0,256):
            payload = padding[len(cleartext):(size-1)] + cleartext + int.to_bytes(j,1,'little') #produces the blocks formatted as 'A'*x + Known_bytes + Test_byte
            output = encryption_oracle_ecb(payload,secret_key) 
            dictionary[output[:size]] = payload #saves in a dictionary so that search the output and get the input

        payload = b'A'*(size-len(cleartext)-1) #this is a block in the format of 'A'*x. So that it completes with the text I want to know in the oracle
        output = encryption_oracle_ecb(payload,secret_key)
        try:
            cleartext = dictionary[output[shift:shift+size]][size-len(cleartext)-1:] # the shift is so that it works on blocks other than the first one
        except:
            continue
    
    return cleartext

padding = b'A'*size
cyphertext = encryption_oracle_ecb(b'',secret_key)
blocks = {}

for i in range(0,round(len(cyphertext)/size)):
    blocks[i] = decrypt_block(16,i,padding[1:])
    padding = blocks[i]
    
b''.join(list(blocks.values()))

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"

## 13-ECB cut-and-paste

Write a k=v parsing routine, as if for a structured cookie. The routine should take:
```
foo=bar&baz=qux&zap=zazzle
```
... and produce:
```
{
  foo: 'bar',
  baz: 'qux',
  zap: 'zazzle'
}
```
(you know, the object; I don't care if you convert it to JSON).

Now write a function that encodes a user profile in that format, given an email address. You should have something like:
```
profile_for("foo@bar.com")
```
... and it should produce:
```
{
  email: 'foo@bar.com',
  uid: 10,
  role: 'user'
}
```
... encoded as:
```
email=foo@bar.com&uid=10&role=user
```
Your "profile_for" function should not allow encoding metacharacters (& and =). Eat them, quote them, whatever you want to do, but don't let people set their email address to "foo@bar.com&role=admin".

Now, two more easy functions. Generate a random AES key, then:

1. Encrypt the encoded user profile under the key; "provide" that to the "attacker".
2. Decrypt the encoded user profile and parse it.

Using only the user input to profile_for() (as an oracle to generate "valid" ciphertexts) and the ciphertexts themselves, make a role=admin profile.


In [14]:
## Challenge 13 ECB cut-and-paste
# This one is pretty fun

#firstly I make the parser
def parser_string_cookie(input1,separator):
    variables = input1.decode('ascii').split(separator)
    output = {}
    for variable in variables:
        output[variable.split("=")[0]] = variable.split("=")[1]
    
    return output

#test to see if the parser works
input1 = "foo=bar&baz=qux&zap=zazzle"
#parser_string_cookie(input1)

def profile_for(email):
    attributes = {}
    attributes['email'] = email.replace('&','').replace('=','')
    attributes['uid'] = 10;
    attributes['role'] = 'user'
    
    aux = []
    for attribute in attributes.keys():
        aux.append('='.join([attribute,str(attributes[attribute])]))
        
    return '&'.join(aux)

def encrypt_user(email,key):
    cleartext = profile_for(email);
    algoritmo = AES.new(key,AES.MODE_ECB);
    cyphertext = algoritmo.encrypt(pkcs_padding_long(bytes(cleartext,'ascii'),16));
    
    return cyphertext

def decrypt_user(user_encrypted,key):
    algoritmo = AES.new(key,AES.MODE_ECB);
    cleartext = algoritmo.decrypt(user_encrypted);
    return parser_string_cookie(pkcs_unpadding_long(cleartext,16),"&")
    
random_key = Random.get_random_bytes(16);

cypheruser = encrypt_user('hcleves',random_key)

# Find the encryption key. It's exactly the same code I made in challenge 12
previous = len(encryption_oracle_ecb(b'',secret_key))
for i in range(1,120): # 120 is just an arbitrary maximum I chose
    input1 = b'A'*i;
    output = encryption_oracle_ecb(input1,secret_key) #secret_key has to stay consistent between interations, so that's why I declared as a global variable in the first block of code
    if len(output)!=previous: 
        size = len(output)-previous
        break

## the message is like "email=<PAYLOAD>&uid=10&role=user".The idea is putting "email=" on the first block
## the second block is our payload
## the last block I want to be only the "user", so that I can change by the "admin" block, which I'm gonna found out
## using the second block
## Do not forget the pcks padding, which is going to be present in the last block

# The first plaintext that will be encrypted is:
# block 1:           block 2 (pkcs7 padded):                             block 3:               block 4 (ommiting the padding):
# email=AAAAAAAAAA   admin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b   AAA&uid=10&role=       user
payload = 'A'*(size-len("admin=")) + pkcs_padding_long(b'admin',size).decode('ascii') + 'A'*(size-len('&uid=10&role='))
cypheruser = encrypt_user(payload,random_key)
cypheruser = cypheruser[:-size] + cypheruser[size:size*2] #here I replace the last block with the second one
decrypt_user(cypheruser,random_key)

{'email': 'AAAAAAAAAAadmin\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0b\x0bAAA',
 'uid': '10',
 'role': 'admin'}

## 14-Byte-at-a-time ECB decryption (Harder)

Take your oracle function from #12. Now generate a random count of random bytes and prepend this string to every plaintext. You are now doing:
```
AES-128-ECB(random-prefix || attacker-controlled || target-bytes, random-key)
```
Same goal: decrypt the target-bytes.

>Stop and think for a second.
>
>What's harder than challenge #12 about doing this? How would you overcome that obstacle? The hint is: you're using all the tools you already have; no crazy math is required.
>
>Think "STIMULUS" and "RESPONSE".


In [15]:
## Challenge 14 Byte-at-a-time ECB decryption (Harder)

def encryption_oracle_ecb_prefix(cleartext,key):
    text_to_prepend = Random.get_random_bytes(rnd.randint(0,16));
    text_to_append = "Um9sbGluJyBpbiBteSA1LjAKV2l0aCBteSByYWctdG9wIGRvd24gc28gbXkgaGFpciBjYW4gYmxvdwpUaGUgZ2lybGllcyBvbiBzdGFuZGJ5IHdhdmluZyBqdXN0IHRvIHNheSBoaQpEaWQgeW91IHN0b3A/IE5vLCBJIGp1c3QgZHJvdmUgYnkK"
    text_to_append = codecs.decode(bytes(text_to_append,'ascii'),'base64')
    cleartext = text_to_prepend + cleartext + text_to_append;
    algoritmo = AES.new(key,AES.MODE_ECB);
    cyphertext = algoritmo.encrypt(pkcs_padding_long(cleartext,16));
    
    return cyphertext

## Strategy: Firstly, I encrypt an block to function as an identifier, for example, a block full of 'A's only
## To achieve that, I just need to input the oracle a really long sequency of A's and look at the blocks that repeat
## Then, I can use this block full of A's as an identifier
## After I located what encrypted block correspond as an block full of A's I know exactly the position of the 
## target-bytes in the cryptotext. So, the prefix is not an nuisance anymore and I can discover the target-bytes
## using the same method as the challenge 12
## The real problem is that I don't know the size of the random-prefix, but I can just assume one size and then
## call the oracle enough times that some calls will have that size.

#adds the identifier
def payload_encapsule(payload): 
    identifier = b'A'*16;
    message = b''
    for i in range(0,16):
        message += int.to_bytes(i,1,'little')*i + identifier + payload
    return message

## I have to find the value of the encrypted identifier
identifier = b'A'*16;

def most_frequent(List): 
    occurence_count = Counter(List) 
    return occurence_count.most_common(1)[0][0]

cyphertext = encryption_oracle_ecb_prefix(b'A'*1600,secret_key)
pieces = [cyphertext[i:i+len(secret_key)] for i in range(0,len(cyphertext),len(secret_key))]
identifier = most_frequent(pieces)
#so there we have it

# the padding here is with the same meaning as the challenge 12
padding = b'B'*16;
def decrypt_block_prefix(size,block_index,padding,identifier):
    cleartext = b''
    for i in range(0,size):
        dictionary = {}
        for j in range(0,256):
            payload = padding[len(cleartext):(size-1)] + cleartext + int.to_bytes(j,1,'little')
            cyphertext = encryption_oracle_ecb_prefix(payload_encapsule(payload),secret_key)

            pieces = [cyphertext[a:a+len(secret_key)] for a in range(0,len(cyphertext),len(secret_key))]
            payload_encrypted = pieces[pieces.index(identifier)+1] #next block after identifier
            dictionary[payload_encrypted] = payload

        payload = b'w'*7 + b'A'*size + b'B'*(size-len(cleartext)-1) ## I choose 7 arbitrarily, any integer between 0-16 would work
        cyphertext = encryption_oracle_ecb_prefix(payload,secret_key)
        pieces = [cyphertext[a:a+len(secret_key)] for a in range(0,len(cyphertext),len(secret_key))]
    
        #the counter is there only to prevent the loop from taking too long
        counter = 0
        while (identifier not in pieces) and counter<1000: ## keeps calling the oracle until the prefix has the length I desire
            cyphertext = encryption_oracle_ecb_prefix(payload,secret_key)
            pieces = [cyphertext[a:a+len(secret_key)] for a in range(0,len(cyphertext),len(secret_key))]
            counter = counter+1;

        if counter>=1000:
            raise NameError('Taking too time in the loop')
        
        payload_encrypted = pieces[pieces.index(identifier)+1+block_index]
        try:
            cleartext = dictionary[payload_encrypted][size-len(cleartext)-1:]
        except:
            continue
    
    return cleartext
    
padding = b'B'*size
cyphertext = encryption_oracle_ecb_prefix(b'',secret_key)
blocks = {}
for i in range(0,round(len(cyphertext)/size)-1): 
    blocks[i] = decrypt_block_prefix(16,i,padding[1:],identifier)
    padding = blocks[i]
    
b''.join(list(blocks.values()))

b"Rollin' in my 5.0\nWith my rag-top down so my hair can blow\nThe girlies on standby waving just to say hi\nDid you stop? No, I just drove by\n\x01"

## 15-PKCS#7 padding validation

Write a function that takes a plaintext, determines if it has valid PKCS#7 padding, and strips the padding off.

The string:
```
"ICE ICE BABY\x04\x04\x04\x04"
```
... has valid padding, and produces the result "ICE ICE BABY".

The string:
```
"ICE ICE BABY\x05\x05\x05\x05"
``` 
... does not have valid padding, nor does:
``` 
"ICE ICE BABY\x01\x02\x03\x04"
```
If you are writing in a language with exceptions, like Python or Ruby, make your function throw an exception on bad padding.

Crypto nerds know where we're going with this. Bear with us.

In [16]:
## Challenge 15 PKCS#7 padding validation 

def reverse_byte_order(string):
    return bytes([string[a] for a in range(len(string)-1,-1,-1)])

def strip_padding_with_check(string,size,debug=False):
    if(len(string)%size!=0):
        return string
    
    string_reverse = reverse_byte_order(string)
    candidate = string_reverse[0];
    padding = string_reverse[:int(candidate)]
    set_padding = set(padding)
    
    if padding == int.to_bytes(candidate,1,'little')*int(candidate) and candidate!=0:
        return pkcs_unpadding_long(string,size)
        
    else:
        raise ValueError('String is not padded')

test = b"ICE ICE BABY\x04\x04\x04\x04"
strip_padding_with_check(test,16)

b'ICE ICE BABY'

## 16-CBC bitflipping attacks

Generate a random AES key.

Combine your padding code and CBC code to write two functions.

The first function should take an arbitrary input string, prepend the string:
```
"comment1=cooking%20MCs;userdata="
```
... and append the string:
```
";comment2=%20like%20a%20pound%20of%20bacon"
```
The function should quote out the ";" and "=" characters.

The function should then pad out the input to the 16-byte AES block length and encrypt it under the random AES key.

The second function should decrypt the string and look for the characters ";admin=true;" (or, equivalently, decrypt, split the string on ";", convert each resulting string into 2-tuples, and look for the "admin" tuple).

Return true or false based on whether the string exists.

If you've written the first function properly, it should not be possible to provide user input to it that will generate the string the second function is looking for. We'll have to break the crypto to do that.

Instead, modify the ciphertext (without knowledge of the AES key) to accomplish this.

You're relying on the fact that in CBC mode, a 1-bit error in a ciphertext block:

* Completely scrambles the block the error occurs in
* Produces the identical 1-bit error(/edit) in the next ciphertext block.

>Stop and think for a second.
>
>Before you implement this attack, answer this question: why does CBC mode have this property?


In [17]:
## Challenge 16 CBC bitflipping attacks

def escape(s):
    return s.replace("%", "%37").replace("=", "%61").replace(";", "%59")

def unescape(s):
    return s.replace("%59", ";").replace("%61", "=").replace("%37", "%")

def encrypt_user_data(userdata,key,iv):
    userdata = escape(userdata)
    prefix = "comment1=cooking%20MCs;userdata="
    suffix = ";comment2=%20like%20a%20pound%20of%20bacon"
    cleartext = prefix + userdata + suffix
    cleartext = pkcs_padding_long(bytes(cleartext,'ascii'),len(key))
    cyphertext = cbc_encrypt_manual(cleartext,key,iv)
    
    return cyphertext

def decrypt_user_data(cyphertext,key,iv):
    cleartext = cbc_decrypt_manual(cyphertext,key,iv)
    cleartext = strip_padding_with_check(cleartext,len(key))
    
    if b";admin=true;" in cleartext:
        return True;
    else:
        return False;

## Strategy
## Here I wanna change the cleartext of one of the blocks to ";admin=true".  To do that I'm gonna use
## the CBC property that it XOR's the cleartext from an block with the cyphertext from the previous block
## therefore, by altering the cyphertext of the previous block you can alter the cleartext of the next block
## The problem with this approach is that the previous block cleartext becames pretty non-sensical, but since
## the validation only search for an string, it will work

target = b";admin=true;a=77" # the ";a=77" was only added to make the target the length of an block

##the prefix has 32 bytes therefore the payload goes in the first block
payload = 'A'*16 + 'B'*16
cyphertext = encrypt_user_data(payload,secret_key,iv_secret)
block = cyphertext[32:48]

cyphertext = cyphertext[:32] + xor_string(target,xor_string(b'B'*16,block)) + cyphertext[48:]
decrypt_user_data(cyphertext,secret_key,iv_secret)

True

# Crypto Challenge Set 3

This is the next set of block cipher cryptography challenges (even the randomness stuff here plays into block cipher crypto).

This set is moderately difficult. It includes a famous attack against CBC mode, and a "cloning" attack on a popular RNG that can be annoying to get right.

We've also reached a point in the crypto challenges where all the challenges, with one possible exception, are valuable in breaking real-world crypto. 

## 17-The CBC padding oracle

This is the best-known attack on modern block-cipher cryptography.

Combine your padding code and your CBC code to write two functions.

The first function should select at random one of the following 10 strings:
```
MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=
MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=
MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==
MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==
MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl
MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==
MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==
MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=
MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=
MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93
```
... generate a random AES key (which it should save for all future encryptions), pad the string out to the 16-byte AES block size and CBC-encrypt it under that key, providing the caller the ciphertext and IV.

The second function should consume the ciphertext produced by the first function, decrypt it, check its padding, and return true or false depending on whether the padding is valid.

>What you're doing here.
>
>This pair of functions approximates AES-CBC encryption as its deployed serverside in web applications; the second function models the server's consumption of an encrypted session token, as if it was a cookie.

It turns out that it's possible to decrypt the ciphertexts provided by the first function.

The decryption here depends on a side-channel leak by the decryption function. The leak is the error message that the padding is valid or not.

You can find 100 web pages on how this attack works, so I won't re-explain it. What I'll say is this:

The fundamental insight behind this attack is that the byte 01h is valid padding, and occur in 1/256 trials of "randomized" plaintexts produced by decrypting a tampered ciphertext.

02h in isolation is not valid padding.

02h 02h is valid padding, but is much less likely to occur randomly than 01h.

03h 03h 03h is even less likely.

So you can assume that if you corrupt a decryption AND it had valid padding, you know what that padding byte is.

It is easy to get tripped up on the fact that CBC plaintexts are "padded". Padding oracles have nothing to do with the actual padding on a CBC plaintext. It's an attack that targets a specific bit of code that handles decryption. You can mount a padding oracle on any CBC block, whether it's padded or not.


Note: This block from now on is not a part of the question. It's a comment of mine.

I found this problem to be very difficult. Definitely the most difficult so far. I spend a large amount of time trying it and just couldn't get it right. So, I search the web for answers so that I could at least understand the logic of someone else who solved it. 

And I came across this explanation from [Cédric Van Rompay](https://cedricvanrompay.gitlab.io/cryptopals/challenges/17.html) that really helped me. Here's the part that helped me,although I highly recommend you check his original site, there's an link there to the original paper that discribed this kind of attack.

>As we already did, we will take an example with a block size of 4 just for the sake of simplicity.
>
>he message will be 6 characters encoded as 6 bytes (it could be the word “attack” for instance), and noted M1 M2 M3 M4 M5 M6. PKCS#7 padding for this message with a block size of 4 requires to add two bytes of padding, so the padded messa eg will be
>
>M1 M2 M3 M4 | M5 M6 P P
>
>Where a "pipe" character | denotes the block limits and where the P byte is equal to 0x02
>
>Here is how we can recover the last message byte M6 using a "padding oracle", that is some external way to know if the padding if the underlying message is correct or not.
>
>Remember with CBC mode we are able to flip bits in the underlying plaintext by flipping bits in the ciphertext. This makes us capable of XORing the plaintext with an arbitrary value (XORing is equivalent to bit flipping) as long as we are only affecting bytes in a same block. We will do this using the cbc_xor function which we created in the previous challenges and put in our libmatasano.py file.
>
>First of all this gives us a very simple way to tell how many bytes of PKCS#7 padding bytes there are at the end of the plaintext: You use CBC bitflipping to alter the very last byte. It is very probable that the resulting message will have an invalind padding.
>
>M1 M2 M3 M4 | M5 M6 P  P
>⊕ X = M1 M2 M3 M4 | M5 M6 P P'
>
>unless P' = 1, oracle will give a Padding Error
>
>To remove the probability of not getting a padding error when you are altering a padding byte, you can just re-test with a different value X: you know the byte you are altering is part of the padding if you got a padding error with at least one value of X.
>
>You then alter the second-to-last byte of plaintext, see if the padding oracle raises a padding error, and so on. When you don't get a padding error whatever value you use for X, it means that the byte you are altering was not part of the padding.
>
>M1 M2 M3 M4 | M5 M6 P  P
>⊕ X
>= M1 M2 M3 M4 | M5 ?? P P
>
>no padding error, whatever the value of 'X' is
>
>The technique to recover the message bytes is quite similar.
>
>We will use bitflipping in order to perform the following operation:
>
>M1 M2 M3 M4 | M5 M6 P  P
>⊕ X1 X2 X3 = M1 M2 M3 M4 | M5 P' P' P'
>
>Such that the resulting message is properly padded, that is in our example P' = 3.
>
>X2 and X3 are easy to compute: because we know the padding lenght we know the values of the original padding bytes P, and we know the value of the padding bytes P' we want to get after bit flipping (P' = P+1). Thus, it is easy for us to compute X2 = X3 = P ⊕ P'.
>
>What we don't know is which value of X1 will result in a proper padding. We just have to try every single value and give the result to the padding oracle: when we don't get a padding error, it means that we found the proper byte X1.
>
>M1 M2 M3 M4 | M5 M6 P  P
>⊕ i X2 X3 = M1 M2 M3 M4 | M5 ?? P' P'
>
>Padding Error unless M6 ⊕ i = P', that is, i = X1
>
>It is now easy to infer the value of M6:
>
>M6 = P' ⊕ X1
>
>We can then repeat the same process to learn message byte M5, trying to find new values X1 X2 X3 X4 such that
>
>M1 M2 M3 M4 | M5 M6 P  P
>⊕ X1 X2 X3 X4 = M1 M2 M3 M4 | P' P' P' P'
>
>Where P' is equal to 0x04 this time in our example. Again here we will only have to try every posibility for X1 because we already know the proper values for X2 X3 X4 (recall we know M6 now)
>
>Then, there are a couple details to pay attention to to be able to recover the whole message in every situation: When we know a whole block of plaintext, we have to remove it because the paddig oracle will never look at bytes in the second-to-last block.
>
>Say we already recovered M6 and M5, now we are looking for X1 such that the following operation results in a message with proper padding:
>
>M1 M2 M3 M4
>⊕ X1 = M1 M2 M3 P'
>
>(that is we want to obtain P' = 1).
>
>How do we do that ? Same as before: we try every possible value for X1 and give the result to the padding oracle. If we don't get a padding error, it can be one of the following situations:
>
>* we got a last byte of value 1: this is what we wanted
>* we got a last byte of value 2 and it happens that the message byte M3 has a value of 2 as well
>* we got a last byte of value 3 and it happens that message bytes M2 and M3 both have a value of 3 as well
>etc …
>What do we do to make sure we are in the first case and not in the other ones? Well first the first case is much more likely than the others, so we could just accept to have an attack that will not work every now and then.
>
>(TODO mechanism to ensure 100% attack success rate)

In [18]:
## Challenge 17 - cbc padding oracle
strings = ["MDAwMDAwTm93IHRoYXQgdGhlIHBhcnR5IGlzIGp1bXBpbmc=",
"MDAwMDAxV2l0aCB0aGUgYmFzcyBraWNrZWQgaW4gYW5kIHRoZSBWZWdhJ3MgYXJlIHB1bXBpbic=",
"MDAwMDAyUXVpY2sgdG8gdGhlIHBvaW50LCB0byB0aGUgcG9pbnQsIG5vIGZha2luZw==",
"MDAwMDAzQ29va2luZyBNQydzIGxpa2UgYSBwb3VuZCBvZiBiYWNvbg==",
"MDAwMDA0QnVybmluZyAnZW0sIGlmIHlvdSBhaW4ndCBxdWljayBhbmQgbmltYmxl",
"MDAwMDA1SSBnbyBjcmF6eSB3aGVuIEkgaGVhciBhIGN5bWJhbA==",
"MDAwMDA2QW5kIGEgaGlnaCBoYXQgd2l0aCBhIHNvdXBlZCB1cCB0ZW1wbw==",
"MDAwMDA3SSdtIG9uIGEgcm9sbCwgaXQncyB0aW1lIHRvIGdvIHNvbG8=",
"MDAwMDA4b2xsaW4nIGluIG15IGZpdmUgcG9pbnQgb2g=",
"MDAwMDA5aXRoIG15IHJhZy10b3AgZG93biBzbyBteSBoYWlyIGNhbiBibG93"]

def cbc_padding_oracle(key,iv):
    choice = rnd.randint(0,len(strings)-1)
    cleartext = codecs.decode(strings[choice].encode('ascii'),'base64')
    cleartext = pkcs_padding_long(cleartext,len(key))
    cyphertext = cbc_encrypt_manual(cleartext,key,iv)
    return [cyphertext,iv]

def cbc_unpadding_oracle(cyphertext,key,iv,debug=False):
    cleartext = cbc_decrypt_manual(cyphertext,key,iv,padding=False)
    return strip_padding_with_check(cleartext,len(key),debug=debug)

def decipher_block(block,previous):
    payload = bytearray(previous)
    result = bytearray([]) #this is filled from the last byte to first
    
    for i in range(1,len(block)+1):
        suitable = [] #I need an array because there are 2 different situations, I find the byte X1 and I found the byte M6,which was the byte supposed to XOR in normal operation
        calculated = xor_string(result,int.to_bytes(i,1,'little')*i) #this is the calculated n-th last bytes of the block
        
        for j in range(0,256):
            candidate = int.to_bytes(j,1,'little')
            cyphertext = bytes(payload[:-i]) + candidate + reverse_byte_order(calculated) + block
            try:
                cbc_unpadding_oracle(cyphertext,secret_key,previous) 
                suitable.append(candidate) ## if the oracle didn't raise an error, we found the byte X1
            except:
                pass
            
        if(len(suitable)==2): #remove the original byte, so the array only keeps the X1 and we can get it right all the time
            suitable.remove(int.to_bytes(payload[-i],1,'little'))
            
        if(len(suitable)==1): 
            result+= bytearray([int.from_bytes(suitable[0],'little')^i]) 
        else: #this was used only for debugging
            raise Exception('suitable!=1')
        
    return xor_string(previous[-16:],reverse_byte_order(result))

# I use this piece of code quite frequently, so I decided to make a function of it
def break_in_pieces(cyphertext,size):
    pieces = [cyphertext[a:a+size] for a in range(0,len(cyphertext),size)]
    return pieces

output = cbc_padding_oracle(secret_key,iv_secret)
cyphertext = output[0]
pieces = break_in_pieces(cyphertext,16)
cleartext = b''
for i in range(0,len(pieces)):
    if i ==0:
        cleartext += decipher_block(pieces[i],output[1])
    else:
        cleartext += decipher_block(pieces[i],b''.join(pieces[:i]))
    
cleartext

b'000002Quick to the point, to the point, no faking\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f\x0f'

## 18-Implement CTR, the stream cipher mode

The string:
```
L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ==
```
... decrypts to something approximating English in CTR mode, which is an AES block cipher mode that turns AES into a stream cipher, with the following parameters:
```
key=YELLOW SUBMARINE
nonce=0
format=64 bit unsigned little endian nonce,
       64 bit little endian block count (byte count / 16)
``` 
CTR mode is very simple.

Instead of encrypting the plaintext, CTR mode encrypts a running counter, producing a 16 byte block of keystream, which is XOR'd against the plaintext.

For instance, for the first 16 bytes of a message with these parameters:
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00")
```
... for the next 16 bytes:
```
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x01\x00\x00\x00\x00\x00\x00\x00")
```
... and then:
``` 
keystream = AES("YELLOW SUBMARINE",
                "\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00")
```
CTR mode does not require padding; when you run out of plaintext, you just stop XOR'ing keystream and stop generating keystream.

Decryption is identical to encryption. Generate the same keystream, XOR, and recover the plaintext.

Decrypt the string at the top of this function, then use your CTR function to encrypt and decrypt other things.
>This is the only block cipher mode that matters in good code.
>
>Most modern cryptography relies on CTR mode to adapt block ciphers into stream ciphers, because most of what we want to encrypt is better described as a stream than as a sequence of blocks. Daniel Bernstein once quipped to Phil Rogaway that good cryptosystems don't need the "decrypt" transforms. Constructions like CTR are what he was talking about.

In [19]:
## Challenge 18 CTR, the stream cipher mode

encrypted_string = "L77na/nrFsKvynd6HzOoG7GHTLXsTVu9qvY/2syLXzhPweyyMTJULu/6/kXX0KSvoOLSFQ=="

def encrypt_ctr(cleartext,key,iv):
    if len(iv)!=len(key)/2:
        raise Exception('iv must be half the length of key')
    algo = AES.new(key,AES.MODE_ECB)
    pieces = break_in_pieces(cleartext,len(key))
    aux = b''
    for counter in range(0,len(pieces)):
        payload = iv + int.to_bytes(counter,int(len(secret_key)/2),'little')
        aux += algo.encrypt(payload)
    
    return xor_string(cleartext,aux)

def decrypt_ctr(cyphertext,key,iv):
    return encrypt_ctr(cyphertext,key,iv)
    
iv = b'\x00'*8
cyphertext = encrypt_ctr(b'Just testing if both encryption and decryption works properly',secret_key,iv)
print(decrypt_ctr(cyphertext,secret_key,iv))
decrypt_ctr(codecs.decode(encrypted_string.encode('ascii'),'base64'),b"YELLOW SUBMARINE",iv)

b'Just testing if both encryption and decryption works properly'


b"Yo, VIP Let's kick it Ice, Ice, baby Ice, Ice, baby "

## 19-Break fixed-nonce CTR mode using substitutions

Take your CTR encrypt/decrypt function and fix its nonce value to 0. Generate a random AES key.

In successive encryptions (not in one big running CTR stream), encrypt each line of the base64 decodes of the following, producing multiple independent ciphertexts:
```
SSBoYXZlIG1ldCB0aGVtIGF0IGNsb3NlIG9mIGRheQ==
Q29taW5nIHdpdGggdml2aWQgZmFjZXM=
RnJvbSBjb3VudGVyIG9yIGRlc2sgYW1vbmcgZ3JleQ==
RWlnaHRlZW50aC1jZW50dXJ5IGhvdXNlcy4=
SSBoYXZlIHBhc3NlZCB3aXRoIGEgbm9kIG9mIHRoZSBoZWFk
T3IgcG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
T3IgaGF2ZSBsaW5nZXJlZCBhd2hpbGUgYW5kIHNhaWQ=
UG9saXRlIG1lYW5pbmdsZXNzIHdvcmRzLA==
QW5kIHRob3VnaHQgYmVmb3JlIEkgaGFkIGRvbmU=
T2YgYSBtb2NraW5nIHRhbGUgb3IgYSBnaWJl
VG8gcGxlYXNlIGEgY29tcGFuaW9u
QXJvdW5kIHRoZSBmaXJlIGF0IHRoZSBjbHViLA==
QmVpbmcgY2VydGFpbiB0aGF0IHRoZXkgYW5kIEk=
QnV0IGxpdmVkIHdoZXJlIG1vdGxleSBpcyB3b3JuOg==
QWxsIGNoYW5nZWQsIGNoYW5nZWQgdXR0ZXJseTo=
QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=
VGhhdCB3b21hbidzIGRheXMgd2VyZSBzcGVudA==
SW4gaWdub3JhbnQgZ29vZCB3aWxsLA==
SGVyIG5pZ2h0cyBpbiBhcmd1bWVudA==
VW50aWwgaGVyIHZvaWNlIGdyZXcgc2hyaWxsLg==
V2hhdCB2b2ljZSBtb3JlIHN3ZWV0IHRoYW4gaGVycw==
V2hlbiB5b3VuZyBhbmQgYmVhdXRpZnVsLA==
U2hlIHJvZGUgdG8gaGFycmllcnM/
VGhpcyBtYW4gaGFkIGtlcHQgYSBzY2hvb2w=
QW5kIHJvZGUgb3VyIHdpbmdlZCBob3JzZS4=
VGhpcyBvdGhlciBoaXMgaGVscGVyIGFuZCBmcmllbmQ=
V2FzIGNvbWluZyBpbnRvIGhpcyBmb3JjZTs=
SGUgbWlnaHQgaGF2ZSB3b24gZmFtZSBpbiB0aGUgZW5kLA==
U28gc2Vuc2l0aXZlIGhpcyBuYXR1cmUgc2VlbWVkLA==
U28gZGFyaW5nIGFuZCBzd2VldCBoaXMgdGhvdWdodC4=
VGhpcyBvdGhlciBtYW4gSSBoYWQgZHJlYW1lZA==
QSBkcnVua2VuLCB2YWluLWdsb3Jpb3VzIGxvdXQu
SGUgaGFkIGRvbmUgbW9zdCBiaXR0ZXIgd3Jvbmc=
VG8gc29tZSB3aG8gYXJlIG5lYXIgbXkgaGVhcnQs
WWV0IEkgbnVtYmVyIGhpbSBpbiB0aGUgc29uZzs=
SGUsIHRvbywgaGFzIHJlc2lnbmVkIGhpcyBwYXJ0
SW4gdGhlIGNhc3VhbCBjb21lZHk7
SGUsIHRvbywgaGFzIGJlZW4gY2hhbmdlZCBpbiBoaXMgdHVybiw=
VHJhbnNmb3JtZWQgdXR0ZXJseTo=
QSB0ZXJyaWJsZSBiZWF1dHkgaXMgYm9ybi4=
```
(This should produce 40 short CTR-encrypted ciphertexts).

Because the CTR nonce wasn't randomized for each encryption, each ciphertext has been encrypted against the same keystream. This is very bad.

Understanding that, like most stream ciphers (including RC4, and obviously any block cipher run in CTR mode), the actual "encryption" of a byte of data boils down to a single XOR operation, it should be plain that:
```
CIPHERTEXT-BYTE XOR PLAINTEXT-BYTE = KEYSTREAM-BYTE
```
And since the keystream is the same for every ciphertext:
```
CIPHERTEXT-BYTE XOR KEYSTREAM-BYTE = PLAINTEXT-BYTE (ie, "you don't
say!")
```
Attack this cryptosystem piecemeal: guess letters, use expected English language frequence to validate guesses, catch common English trigrams, and so on.

>Don't overthink it.
>
>Points for automating this, but part of the reason I'm having you do this is that I think this approach is suboptimal.


In [20]:
##Challenge 19 - Break fixed-nonce CTR mode using substitutions
# This challenge here is just to play around with the encrypted cyphertexts
# I don't think I'm gonna publish the code with my playthrough on this challenge cause it's pretty messy
# And the more organized trial is basically the answer to challenge 20.
# But I'm gonna list some tools that could help
# https://crypto.interactive-maths.com/frequency-analysis-breaking-the-code.html
# https://www.dcode.fr/frequency-analysis

## 20-Break fixed-nonce CTR statistically

In this [file](20.txt) find a similar set of Base64'd plaintext. Do with them exactly what you did with the first, but solve the problem differently.

Instead of making spot guesses at to known plaintext, treat the collection of ciphertexts the same way you would repeating-key XOR.

Obviously, CTR encryption appears different from repeated-key XOR, but with a fixed nonce they are effectively the same thing.

To exploit this: take your collection of ciphertexts and truncate them to a common length (the length of the smallest ciphertext will work).

Solve the resulting concatenation of ciphertexts as if for repeating- key XOR, with a key size of the length of the ciphertext you XOR'd.


In [21]:
## Challenge 20 Break CTR

file = open('20.txt','r').readlines()
lines = []
# decode lines from base64
for i in range(0,len(file)):
    file[i] = file[i].strip()
    lines.append(codecs.decode(file[i].encode('ASCII'),'base64'))
    
#encrypt lines using CTR from Challenge 18
encrypted_lines=[]
for i in range(0,len(lines)):
    encrypted_lines.append(encrypt_ctr(lines[i],secret_key,8*b'\x00'))
    
sizes = [len(x) for x in encrypted_lines]

# Here I truncate the lines to the size of the smallest one. This is necessary cause I need to have some letters to do frequency analysis.
# An nice idea for later is to try to do this but with less data.
truncated = [line[:min(sizes)] for line in encrypted_lines]

#This is the same functions and idea from challenge 6.
blocks = [GetNthLetters(b''.join(truncated),i,min([len(x) for x in truncated])) for i in range(0,min(sizes))]

key = []
for block in blocks:
    key.append(find_letter_xor(block,byte=True))

for string in truncated:
    print(xor_string(string,bytes(key)))

b'i\'m rated "R"...this is a warning, ya better void / P'
b'cuz I came back to attack others in spite- / Strike l'
b"but don't be afraid in the dark, in a park / Not a sc"
b'ya tremble like a alcoholic, muscles tighten up / Wha'
b'suddenly you feel like your in a horror flick / You g'
b"music's the clue, when I come your warned / Apocalyps"
b"haven't you ever heard of a MC-murderer? / This is th"
b'death wish, so come on, step to this / Hysterical ide'
b'friday the thirteenth, walking down Elm Street / You '
b'this is off limits, so your visions are blurry / All '
b"terror in the styles, never error-files / Indeed I'm "
b'for those that oppose to be level or next to this / I'
b"worse than a nightmare, you don't have to sleep a win"
b'flashbacks interfere, ya start to hear: / The R-A-K-I'
b'then the beat is hysterical / That makes Eric go get '
b'soon the lyrical format is superior / Faces of death '
b"mC's decaying, cuz they never stayed / The scene of a"
b"the fiend of a rhyme on the 

## 21-Implement the MT19937 Mersenne Twister RNG

You can get the psuedocode for this from Wikipedia.

If you're writing in Python, Ruby, or (gah) PHP, your language is probably already giving you MT19937 as "rand()"; don't use rand(). Write the RNG yourself.


In [22]:
## Challenge 21 implement the MT19937 Mersenne Twister RNG
## This one is a bit boring, because it's basically just follow the pseudocode specs on Wikipedia
## Pseudocode can be found on https://en.wikipedia.org/wiki/Mersenne_Twister

#int need to be of size 64
def generate_x_bits_1(x):
        a=0
        for i in range(0,x):
            a +=pow(2,i)
        return a

class MyRandom():
    def __init__(self):
        (self.w, self.n, self.m, self.r) = (32, 624, 397, 31)
        self.a = 0x9908B0DF
        (self.u, self.d) = (11, 0xFFFFFFFF)
        (self.s, self.b) = (7, 0x9D2C5680)
        (self.t, self.c) = (15, 0xEFC60000)
        self.l = 18
        self.f = 1812433253

        # Create a length n array to store the state of the generator
        self.MT = [int(0)]*self.n #int array
        self.index = self.n+1 #int

        self.lower_mask = (1 << self.r) - 1 #const int r bits of 1 
        #const int upper_mask = lowest w bits of (not lower_mask) //r bits of 0 followed by w-r bits of 1
        self.upper_mask = (generate_x_bits_1(self.w))&(~self.lower_mask)
    
    #Initialize the generator from a seed
    def seed_mt(self,seed):
        '''Initialize the generator from a seed'''
        self.index = self.n
        self.MT[0] = seed
        #for i from 1 to (n - 1) { // loop over each element
        for i in range(1,self.n):
            #MT[i] := lowest w bits        of (f * (MT[i-1] xor (MT[i-1] >> (w-2))) + i)
            self.MT[i] = (generate_x_bits_1(self.w)) &  (self.f * (self.MT[i-1] ^   (self.MT[i-1] >> (self.w-2))) + i)

    def twist(self):
        '''Generate the next n values from the series x_i '''
        for i in range(0,self.n):
            x = ( self.MT[i] & self.upper_mask ) + ( self.MT[(i+1)%self.n] & self.lower_mask )
            xA = x >> 1
            if x % 2 != 0:
                xA = xA ^ self.a
            self.MT[i] = self.MT[(i+self.m)%self.n] ^ xA
            
        self.index = 0
        return self.MT

    def extract_number(self):
        ''' Extract a tempered value based on MT[index]
        calling twist() every n numbers'''
        if self.index>=self.n:
            if self.index>self.n:
                raise Exception('Generator was never seeded')
                #Alternatively, seed with constant value; 5489 is used in reference C code[51]
            self.twist()
        
        y = self.MT[self.index]
        y = y ^ ((y >> self.u) & self.d)
        y = y ^ ((y << self.s) & self.b)
        y = y ^ ((y << self.t) & self.c)
        y = y ^ (y >> self.l)

        self.index = self.index + 1
        return generate_x_bits_1(self.w)&y
    
        
random_obj = MyRandom()
random_obj.seed_mt(1)
random_obj.extract_number()

1791095845

## 22-Crack an MT19937 seed

Make sure your MT19937 accepts an integer seed value. Test it (verify that you're getting the same sequence of outputs given a seed).

Write a routine that performs the following operation:

* Wait a random number of seconds between, I don't know, 40 and 1000.
* Seeds the RNG with the current Unix timestamp
* Waits a random number of seconds again.
* Returns the first 32 bit output of the RNG.

You get the idea. Go get coffee while it runs. Or just simulate the passage of time, although you're missing some of the fun of this exercise if you do that.

From the 32 bit RNG output, discover the seed.

In [27]:
## Challenge 22 break the seed of the RNG
# This is an pretty well known attack that I think it's used somehow
# Cause I am seeding the random generator with an timestamp and I know roughly when it happened,
# I can just seed a copy of it with all the value of the timestamp within an certain window of time, which is
# small enough that we can guess them all, and by comparing the output we can tell the right one

#get current time
def time_oracle(): 
    current_time = int(time.time())
    ## Ain't nobody got time to wait
    wait_1 = current_time + rnd.randint(40,1000)
    random_obj = MyRandom()
    random_obj.seed_mt(wait_1)
    wait_2 = wait_1 + rnd.randint(40,1000)
    return random_obj.extract_number()

output = time_oracle()

current_time = int(time.time()) + 2000

#here we guess all the timestamps, so we can find the right one.
#I used the disabled flag cause it takes a while to run
enable=False
if enable:
    for i in range(current_time,(current_time)-(2*1000+50),-1):
        random_obj.seed_mt(i)
        if random_obj.extract_number() == output:
            print('Found the timestamp', i)
            break;
    else:
        print('It didn`t work')